In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14128,2021-12-27T17:00:00,ITA,19,Sicilia,38.115697,13.362357,657,81,738,29126,...,DECEDUTI: N. 2 IL 26/12/21 - N. 12 IL 25/12/21...,7.0,NaN,NaN,356496.0,0.0,3755617.0,4426873.0,ITG,ITG1
14129,2021-12-27T17:00:00,ITA,9,Toscana,43.769231,11.255889,493,76,569,31704,...,NaN,6.0,NaN,NaN,330322.0,6804.0,5353615.0,3525975.0,ITI,ITI1
14130,2021-12-27T17:00:00,ITA,10,Umbria,43.106758,12.388247,93,9,102,8405,...,NaN,0.0,NaN,NaN,76610.0,422.0,1324223.0,1474900.0,ITI,ITI2
14131,2021-12-27T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,31,2,33,1202,...,NaN,1.0,NaN,NaN,13050.0,2014.0,124109.0,220821.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14128,2021-12-27T17:00:00,19,Sicilia,657,81,738,29126,29864,1335,2087,...,356496,8182490,2949655.0,7.0,356496.0,0.0,3755617.0,4426873.0,ITG,ITG1
14129,2021-12-27T17:00:00,9,Toscana,493,76,569,31704,32273,2283,2843,...,337126,8879590,3581943.0,6.0,330322.0,6804.0,5353615.0,3525975.0,ITI,ITI1
14130,2021-12-27T17:00:00,10,Umbria,93,9,102,8405,8507,886,935,...,77032,2799123,493535.0,0.0,76610.0,422.0,1324223.0,1474900.0,ITI,ITI2
14131,2021-12-27T17:00:00,2,Valle d'Aosta,31,2,33,1202,1235,-11,93,...,15064,344930,106317.0,1.0,13050.0,2014.0,124109.0,220821.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-12-27', '2021-12-26')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-12-27T17:00:00,1,Piemonte,1058,89,1147,41004,42151,3231,4611,...,454149,11000236,2980291.0,10.0,392245.0,61904.0,4096869.0,6903367.0,ITC,ITC1
1,2021-12-27T17:00:00,2,Valle d'Aosta,31,2,33,1202,1235,-11,93,...,15064,344930,106317.0,1.0,13050.0,2014.0,124109.0,220821.0,ITC,ITC2
2,2021-12-27T17:00:00,3,Lombardia,1539,187,1726,122154,123880,3893,5065,...,1075263,24046523,6651371.0,14.0,925005.0,150258.0,12783197.0,11263326.0,ITC,ITC4
3,2021-12-27T17:00:00,5,Veneto,1104,173,1277,70484,71761,1122,2816,...,610250,19039997,2436402.0,15.0,570070.0,40180.0,8187720.0,10852277.0,ITH,ITH3
4,2021-12-27T17:00:00,6,Friuli Venezia Giulia,284,27,311,8521,8832,-148,154,...,149079,4423912,951888.0,5.0,129670.0,19409.0,2609045.0,1814867.0,ITH,ITH4
5,2021-12-27T17:00:00,7,Liguria,505,40,545,9988,10533,262,469,...,142079,3448997,1027431.0,1.0,142079.0,0.0,1961301.0,1487696.0,ITC,ITC3
6,2021-12-27T17:00:00,8,Emilia-Romagna,1236,107,1343,56376,57719,2897,3482,...,512130,11444857,2368914.0,7.0,511555.0,575.0,7167787.0,4277070.0,ITH,ITH5
7,2021-12-27T17:00:00,9,Toscana,493,76,569,31704,32273,2283,2843,...,337126,8879590,3581943.0,6.0,330322.0,6804.0,5353615.0,3525975.0,ITI,ITI1
8,2021-12-27T17:00:00,10,Umbria,93,9,102,8405,8507,886,935,...,77032,2799123,493535.0,0.0,76610.0,422.0,1324223.0,1474900.0,ITI,ITI2
9,2021-12-27T17:00:00,11,Marche,203,42,245,6581,6826,0,309,...,138766,1946506,1132957.0,6.0,138766.0,0.0,1611964.0,334542.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-12-26T17:00:00,1,Piemonte,995,86,1081,37839,38920,768,1523,...,449538,10949368,2966663.0,9.0,391511.0,58027.0,4090694.0,6858674.0,ITC,ITC1
1,2021-12-26T17:00:00,2,Valle d'Aosta,30,1,31,1215,1246,22,29,...,14971,343629,105922.0,0.0,13021.0,1950.0,123835.0,219794.0,ITC,ITC2
2,2021-12-26T17:00:00,3,Lombardia,1439,180,1619,118368,119987,3306,4581,...,1070198,24002302,6643739.0,14.0,922683.0,147515.0,12769240.0,11233062.0,ITC,ITC4
3,2021-12-26T17:00:00,5,Veneto,1061,160,1221,69418,70639,1253,2093,...,607434,19006349,2433346.0,12.0,568749.0,38685.0,8175825.0,10830524.0,ITH,ITH3
4,2021-12-26T17:00:00,6,Friuli Venezia Giulia,280,29,309,8671,8980,109,246,...,148925,4417407,951267.0,5.0,129566.0,19359.0,2607275.0,1810132.0,ITH,ITH4
5,2021-12-26T17:00:00,7,Liguria,470,41,511,9760,10271,383,643,...,141610,3442593,1026329.0,4.0,141610.0,0.0,1958496.0,1484097.0,ITC,ITC3
6,2021-12-26T17:00:00,8,Emilia-Romagna,1187,107,1294,53528,54822,1996,2921,...,508659,11421614,2365852.0,6.0,508107.0,552.0,7154788.0,4266826.0,ITH,ITH5
7,2021-12-26T17:00:00,9,Toscana,443,72,515,29475,29990,2786,3075,...,334283,8861940,3575921.0,2.0,327554.0,6729.0,5342776.0,3519164.0,ITI,ITI1
8,2021-12-26T17:00:00,10,Umbria,82,9,91,7530,7621,213,348,...,76097,2790185,492446.0,1.0,76097.0,0.0,1321892.0,1468293.0,ITI,ITI2
9,2021-12-26T17:00:00,11,Marche,203,38,241,6585,6826,-42,385,...,138457,1943162,1130580.0,4.0,138457.0,0.0,1610315.0,332847.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-12-27T17:00:00,1,Piemonte,1058,89,1147,41004,42151,3231,4611,...,11000236,2980291.0,10.0,392245.0,61904.0,4096869.0,6903367.0,ITC,ITC1,4.341375e+06
1,2021-12-27T17:00:00,2,Valle d'Aosta,31,2,33,1202,1235,-11,93,...,344930,106317.0,1.0,13050.0,2014.0,124109.0,220821.0,ITC,ITC2,1.255010e+05
2,2021-12-27T17:00:00,3,Lombardia,1539,187,1726,122154,123880,3893,5065,...,24046523,6651371.0,14.0,925005.0,150258.0,12783197.0,11263326.0,ITC,ITC4,1.010397e+07
3,2021-12-27T17:00:00,5,Veneto,1104,173,1277,70484,71761,1122,2816,...,19039997,2436402.0,15.0,570070.0,40180.0,8187720.0,10852277.0,ITH,ITH3,4.907704e+06
4,2021-12-27T17:00:00,6,Friuli Venezia Giulia,284,27,311,8521,8832,-148,154,...,4423912,951888.0,5.0,129670.0,19409.0,2609045.0,1814867.0,ITH,ITH4,1.211357e+06
5,2021-12-27T17:00:00,7,Liguria,505,40,545,9988,10533,262,469,...,3448997,1027431.0,1.0,142079.0,0.0,1961301.0,1487696.0,ITC,ITC3,1.543127e+06
6,2021-12-27T17:00:00,8,Emilia-Romagna,1236,107,1343,56376,57719,2897,3482,...,11444857,2368914.0,7.0,511555.0,575.0,7167787.0,4277070.0,ITH,ITH5,4.467118e+06
7,2021-12-27T17:00:00,9,Toscana,493,76,569,31704,32273,2283,2843,...,8879590,3581943.0,6.0,330322.0,6804.0,5353615.0,3525975.0,ITI,ITI1,3.722729e+06
8,2021-12-27T17:00:00,10,Umbria,93,9,102,8405,8507,886,935,...,2799123,493535.0,0.0,76610.0,422.0,1324223.0,1474900.0,ITI,ITI2,8.802850e+05
9,2021-12-27T17:00:00,11,Marche,203,42,245,6581,6826,0,309,...,1946506,1132957.0,6.0,138766.0,0.0,1611964.0,334542.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-12-27T17:00:00,1,Piemonte,1058,89,1147,41004,42151,3231,4611,...,6903367.0,ITC,ITC1,4.341375e+06,Piemonte,1058,5824,89,628,99
1,2021-12-27T17:00:00,2,Valle d'Aosta,31,2,33,1202,1235,-11,93,...,220821.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,31,99,2,33,0
2,2021-12-27T17:00:00,3,Lombardia,1539,187,1726,122154,123880,3893,5065,...,11263326.0,ITC,ITC4,1.010397e+07,Lombardia,1539,10457,187,1530,0
3,2021-12-27T17:00:00,5,Veneto,1104,173,1277,70484,71761,1122,2816,...,10852277.0,ITH,ITH3,4.907704e+06,Veneto,1104,6000,173,1000,0
4,2021-12-27T17:00:00,6,Friuli Venezia Giulia,284,27,311,8521,8832,-148,154,...,1814867.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,284,1277,27,175,0
5,2021-12-27T17:00:00,7,Liguria,505,40,545,9988,10533,262,469,...,1487696.0,ITC,ITC3,1.543127e+06,Liguria,505,1776,40,219,12
6,2021-12-27T17:00:00,8,Emilia-Romagna,1236,107,1343,56376,57719,2897,3482,...,4277070.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,1236,9001,107,889,0
7,2021-12-27T17:00:00,9,Toscana,493,76,569,31704,32273,2283,2843,...,3525975.0,ITI,ITI1,3.722729e+06,Toscana,493,5033,76,570,27
8,2021-12-27T17:00:00,10,Umbria,93,9,102,8405,8507,886,935,...,1474900.0,ITI,ITI2,8.802850e+05,Umbria,93,662,9,86,41
9,2021-12-27T17:00:00,11,Marche,203,42,245,6581,6826,0,309,...,334542.0,ITI,ITI3,1.518400e+06,Marche,203,980,42,252,51


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-12-27T17:00:00,3,Lombardia,1539,187,1726,122154,123880,3893,5065,...,12.22,3786,484,44221,7632.0,12,1160,11.45,0.05013,10.64
1,2021-12-27T17:00:00,1,Piemonte,1058,89,1147,41004,42151,3231,4611,...,14.17,3165,3088,50868,13628.0,13,1367,9.06,0.10621,10.46
2,2021-12-27T17:00:00,8,Emilia-Romagna,1236,107,1343,56376,57719,2897,3482,...,12.04,2848,561,23243,3062.0,14,560,14.98,0.07795,11.46
3,2021-12-27T17:00:00,12,Lazio,996,135,1131,52274,53405,1848,2933,...,14.32,1814,-732,39551,2291.0,7,1078,7.42,0.05000,8.21
4,2021-12-27T17:00:00,9,Toscana,493,76,569,31704,32273,2283,2843,...,13.33,2229,-232,17650,6022.0,9,551,16.11,0.07637,9.06
5,2021-12-27T17:00:00,5,Veneto,1104,173,1277,70484,71761,1122,2816,...,17.30,1066,723,33648,3056.0,24,1670,8.37,0.05738,12.43
6,2021-12-27T17:00:00,15,Campania,516,36,552,39758,40310,1587,2291,...,6.55,1557,697,30187,4198.0,14,690,7.59,0.03960,9.34
7,2021-12-27T17:00:00,19,Sicilia,657,81,738,29126,29864,1335,2087,...,9.42,1307,360,18154,6890.0,20,732,11.50,0.04201,7.18
8,2021-12-27T17:00:00,10,Umbria,93,9,102,8405,8507,886,935,...,10.47,875,587,8938,1089.0,0,49,10.46,0.10622,8.75
9,2021-12-27T17:00:00,16,Puglia,180,21,201,12001,12202,559,762,...,4.36,551,-121,25090,2927.0,1,202,3.04,0.01901,7.34


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Trento,23,67,90,25.56
1,P.A. Bolzano,19,81,100,19.00
2,Liguria,40,179,219,18.26
3,Veneto,173,827,1000,17.30
4,Marche,42,210,252,16.67
5,Friuli Venezia Giulia,27,148,175,15.43
6,Lazio,135,808,943,14.32
7,Piemonte,89,539,628,14.17
8,Toscana,76,494,570,13.33
9,Calabria,24,163,187,12.83


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))